In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Patch
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'
# 設置微軟正黑體為全域字體
rcParams['font.family'] = 'Microsoft JhengHei'
battery_kWh = 522 * 0.95
battery_kw = 250 * 0.95

In [ ]:
# 讀取 Excel 檔案
file_path = "meter_02293584018_data.xlsx"
raw_data = pd.read_excel(file_path)
time_col = "時間"
usage_col = "用電總量"
default_drop_cols = ['儲冷尖峰', '儲冷半尖峰', '儲冷週六半尖峰', '儲冷離峰', '太陽光電']
raw_data.drop(columns=default_drop_cols, inplace=True, errors='ignore')
# 將時間欄位轉換為日期時間格式
raw_data[time_col] = pd.to_datetime(raw_data[time_col])

# 篩選需要的時間段和欄位
start_time = "15:30:00"
end_time = "21:30:00"
filtered_data = raw_data[
    (raw_data[time_col].dt.time >= pd.to_datetime(start_time).time())
    & (raw_data[time_col].dt.time <= pd.to_datetime(end_time).time())]

# 計算用電總量
columns_to_sum = ['尖峰', '半尖峰', '週六半尖峰', '離峰']
filtered_data[columns_to_sum] = filtered_data[columns_to_sum] * 0.25
filtered_data[usage_col] = filtered_data[columns_to_sum].apply(
    lambda row: row.dropna().unique()[0]
    if row.nunique() == 1 else row.mode().iloc[0],
    axis=1)
filtered_data = filtered_data.drop(columns=columns_to_sum)

In [ ]:
# 按日期統計用電總量
daily_summary = filtered_data.groupby(
    filtered_data[time_col].dt.date)[usage_col].sum()

average_power = daily_summary.mean()

# 設置顏色：夏季（6月16日到10月15日）為橘色，其他為天藍色
colors = [
    'orange' if (date.month > 5 or (date.month == 5 and date.day >= 16)) and
    (date.month < 10 or (date.month == 10 and date.day <= 15)) else 'skyblue'
    for date in daily_summary.index
]

# 繪製柱狀圖
daily_summary.plot(kind='bar',
                   figsize=(16, 7),
                   color=colors,
                   edgecolor=colors,
                   linewidth=1.2,
                   alpha=0.8)

# 設置平均線
plt.axhline(y=average_power, color='red', linestyle='--')

# 設置 X 軸的顯示間隔
x_labels = daily_summary.index
plt.xticks(range(0, len(x_labels), 15),
           x_labels[::15],
           rotation=45,
           fontsize=12)

# 圖表設置
plt.title('尖峰時段 15:30 ~ 21:30 用電總量', fontsize=16)
plt.xlabel('日期', fontsize=14)
plt.ylabel('電量 (kW)', fontsize=14)
plt.xticks(rotation=45, fontsize=12)  # 日期旋轉以便於閱讀

legend_elements = [
    Patch(facecolor='orange', label='夏月尖峰需量'),
    Patch(facecolor='skyblue', label='非夏月尖峰需量'),
    Patch(color='red', label=f'平均值: {average_power:.2f}')
]
plt.legend(handles=legend_elements, fontsize=12, loc='upper right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
plt.close('all')


In [ ]:
date_col = "日期"
hour_col = "小時"
battery_output_power_col = "電池放電功率"
battery_usage_col = "電池使用量"
original_15_max_demand_col = "原始15分鐘尖峰需量"
lower_15_max_demand_col = "調整後15分鐘尖峰需量"

filtered_data[date_col] = filtered_data[time_col].dt.date
filtered_data[hour_col] = filtered_data[time_col].dt.hour
filtered_data[battery_output_power_col] = 0.0

daily_15_demand_reset = filtered_data.groupby([date_col]).apply(
    lambda x: x, include_groups=False).reset_index(drop=True)
daily_15_demand_reset[date_col] = daily_15_demand_reset[time_col].dt.date
daily_15_demand = daily_15_demand_reset.groupby([date_col])

lower_daily_summary = pd.DataFrame(
    columns=[date_col, original_15_max_demand_col, lower_15_max_demand_col])
lower_15_usage = pd.DataFrame(
    columns=[time_col, usage_col, battery_output_power_col])
daily_battery_usage = pd.DataFrame(columns=[date_col, battery_usage_col])
last_index = 0
for (idx, daily_data) in enumerate(daily_15_demand):
    dataframe = daily_data[1]
    current_max_usage = dataframe[usage_col].max()
    current_battery = battery_kWh
    # 弭平高於最低值用電
    min_value = dataframe[usage_col].min()
    for i in range(last_index, last_index + len(dataframe)):
        higher_usage = dataframe.loc[i, usage_col] - min_value
        if higher_usage > 0:
            if higher_usage < current_battery:
                if dataframe.loc[
                        i,
                        battery_output_power_col] + higher_usage * 4 <= battery_kw:
                    dataframe.loc[i, usage_col] = min_value
                    dataframe.loc[i,
                                  battery_output_power_col] += higher_usage * 4
                    current_battery -= higher_usage
                else:
                    output_power = (
                        battery_kw -
                        dataframe.loc[i, battery_output_power_col]) / 4
                    dataframe.loc[i, usage_col] -= output_power
                    dataframe.loc[i, battery_output_power_col] = battery_kw
                    current_battery -= output_power
            else:
                if dataframe.loc[
                        i,
                        battery_output_power_col] + current_battery * 4 <= battery_kw:
                    dataframe.loc[i, usage_col] -= current_battery
                    dataframe.loc[
                        i, battery_output_power_col] += current_battery * 4
                    current_battery = 0
                else:
                    output_power = (
                        battery_kw -
                        dataframe.loc[i, battery_output_power_col]) / 4
                    dataframe.loc[i, usage_col] -= output_power
                    dataframe.loc[i, battery_output_power_col] = battery_kw
                    current_battery -= output_power
    # 均勻降低電量
    if current_battery > 0:
        distribute_battery = current_battery / len(
            dataframe[battery_output_power_col] < battery_kw)
        for i in range(last_index, last_index + len(dataframe)):
            if dataframe.loc[
                    i,
                    battery_output_power_col] + distribute_battery * 4 < battery_kw and dataframe.loc[
                        i, usage_col] > distribute_battery:
                dataframe.loc[i, usage_col] -= distribute_battery
                dataframe.loc[
                    i, battery_output_power_col] += distribute_battery * 4
                current_battery -= distribute_battery
            elif dataframe.loc[
                    i,
                    battery_output_power_col] + distribute_battery * 4 < battery_kw and dataframe.loc[
                        i, usage_col] <= distribute_battery:
                current_battery -= dataframe.loc[i, usage_col]
                dataframe.loc[i, battery_output_power_col] += dataframe.loc[
                    i, usage_col] * 4
                dataframe.loc[i, usage_col] = 0
            elif dataframe.loc[
                    i,
                    battery_output_power_col] + distribute_battery * 4 >= battery_kw:
                output_power = (battery_kw -
                                dataframe.loc[i, battery_output_power_col]) / 4
                dataframe.loc[i, usage_col] -= output_power
                dataframe.loc[i, battery_output_power_col] = battery_kw
                current_battery -= output_power
    for i in range(last_index, last_index + len(dataframe)):
        lower_15_usage.loc[len(lower_15_usage)] = [
            dataframe[time_col].loc[i], dataframe[usage_col].loc[i],
            dataframe[battery_output_power_col].loc[i]
        ]
    last_index += len(dataframe)
    daily_battery_usage.loc[idx] = [
        dataframe[date_col].iloc[0],
        dataframe[battery_output_power_col].sum() * 0.25
    ]
    lower_daily_summary.loc[idx] = [
        dataframe[date_col].iloc[0], current_max_usage * 4,
        dataframe[usage_col].max() * 4
    ]

In [ ]:
# 按日期統計用電總量
original_max_demand_power = lower_daily_summary[
    original_15_max_demand_col].max()
new_max_demand_power = lower_daily_summary[lower_15_max_demand_col].max()

# 設置顏色：夏季（6月16日到10月15日）為橘色，其他為天藍色

new_colors = [
    'lawngreen' if (date.month > 5 or (date.month == 5 and date.day >= 16)) and
    (date.month < 10 or (date.month == 10 and date.day <= 15)) else 'violet'
    for date in lower_daily_summary[date_col]
]
fig, ax = plt.subplots(figsize=(16, 7))

summer_start_index, summer_end_index = 0, 0
for i in range(len(lower_daily_summary[date_col])):
    date = lower_daily_summary[date_col][i]
    if date.month == 5 and date.day == 16:
        summer_start_index = i
    elif date.month == 10 and date.day == 15:
        summer_end_index = i

ax.fill_between(range(len(lower_daily_summary[date_col])),
                lower_daily_summary[original_15_max_demand_col].values,
                label="Daily Summary",
                color='skyblue',
                linewidth=1.2,
                alpha=0.8)
ax.fill_between(range(summer_start_index, summer_end_index),
                lower_daily_summary[original_15_max_demand_col].
                values[summer_start_index:summer_end_index],
                label="Daily Summary",
                color='orange',
                linewidth=1.2,
                alpha=0.8)

# 繪製第二個 DataFrame 的折線圖
ax.fill_between(range(len(lower_daily_summary[date_col])),
                lower_daily_summary[lower_15_max_demand_col].values,
                label="Lower Daily Summary",
                color='violet',
                linewidth=1.2,
                alpha=0.8)

# 設置平均線
ax.axhline(y=original_max_demand_power, color='red', linestyle='--')
ax.axhline(y=new_max_demand_power, color='brown', linestyle='--')

# 設置 X 軸的顯示間隔
x_labels = lower_daily_summary[date_col]
tick_positions = range(0, len(x_labels), max(1, len(x_labels) // 15))  # 設置動態間隔
ax.set_xticks(tick_positions)
ax.set_xticklabels(x_labels[tick_positions], rotation=45, fontsize=12)
ax.set_xlim(left=0, right=len(x_labels))  # X 軸從 0 開始
ax.set_ylim(bottom=0)

# 圖表設置
plt.title('尖峰時段(15:30 ~ 21:30) 15分鐘最高需量', fontsize=16)
plt.xlabel('日期', fontsize=14)
plt.ylabel('最高需量 (kW)', fontsize=14)

legend_elements = [
    Patch(facecolor='orange', label='(原)夏月尖峰需量'),
    Patch(facecolor='skyblue', label='(原)非夏月尖峰需量'),
    Patch(color='red', label=f'(原)尖峰最高需量: {original_max_demand_power:.2f}'),
    Patch(color='brown', label=f'新尖峰最高需量: {new_max_demand_power:.2f}')
]
plt.legend(handles=legend_elements, fontsize=12, loc='upper right')
plt.show()

In [ ]:
# 原始用電情境
original_usage_data = raw_data.copy()
original_usage_data[columns_to_sum] = raw_data[columns_to_sum] * 0.25
original_usage_data[usage_col] = original_usage_data[columns_to_sum].apply(
    lambda row: row.dropna().unique()[0]
    if row.nunique() == 1 else row.mode().iloc[0],
    axis=1)
original_usage_data = original_usage_data.drop(columns=columns_to_sum)
print(original_usage_data)
original_usage_data[date_col] = original_usage_data[time_col].dt.date
original_usage_data[hour_col] = original_usage_data[time_col].dt.hour

# 計算每月電費
# TODO add calculate function

# 更改後用電情境
lower_usage_data = original_usage_data.copy()
updated_df = lower_usage_data.merge(lower_15_usage,
                                    on=time_col,
                                    how='left',
                                    suffixes=('', '_update'))
updated_df[usage_col] = updated_df[usage_col + '_update'].combine_first(
    updated_df[usage_col])
updated_df = updated_df.drop(columns=[usage_col + '_update'])
